# Install the charting library & tables

In [ ]:
!pip install plotly
!pip install tabulate

## Login 

In [ ]:
import requests
import os
import pprint
from dotenv import load_dotenv

# Define the API endpoint and payload
base_url = "https://api-nwfp.rothamsted.ac.uk/api"
endpoint = "auth/v1/login/"
url = f"{base_url}/{endpoint}"

load_dotenv()

payload = {
    "email": os.getenv("API_EMAIL"),
    "password": os.getenv("API_PASSWORD")
}
# Make the POST request
response = requests.post(url, data=payload)

# Check the response
if response.status_code == 200:
    response_data = response.json()
    print("Login successful!")
    token = response_data.get("access")
    headers = {
        "Authorization":f"Bearer {token}",
        "Cache-Control": "no-cache"
    }
    key = token
    print(key)
else:
    print("Failed to login. Status code:", response.status_code)
    print("Response:", response.text)

## Getting help about animals

The NWFP APIs can provide help information, for example if you want to know the available animalCategoryName values we can query using the *Help* parameter. This will tell us what the avalable animalCategoryName values are.

In [ ]:
import requests
import pandas as pd
from tabulate import tabulate

# Define the API endpoint and parameters
endpoint = "animals/v1/getBasicAnimalData/"
api_url = f"{base_url}/{endpoint}"
params = {
    "Help": "animalCategoryName"
}

response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:    
    data = response.json()
    df = pd.DataFrame(data)
    # Reformat the output in a nice table
    print(tabulate(df, headers='keys', tablefmt='grid'))
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)

Now we know one of the animal categories is *Lamb*, we can query demographic data for Lambs using the *animals/v1/getBasicAnimalData/* end point. This example simply prints the returned JSON 

In [ ]:
params = {
    "animalCategoryName":"Lamb",
    "startGrazingYear":2012,
    "endGrazingYear":2012,
    "farmlet":"Red"
}

response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:    
    data = response.json()
    pprint.pp(data)
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)

There is a lot of data! We can use tabulate to present a table of selected columns

In [ ]:
df_results = pd.DataFrame(data.get('results', []))

columns = ["animalid", "officialtag", "managementtag", "breed", "gender", "dateofbirth"]

df_table = df_results[columns]

print(tabulate(df_table, headers='keys', tablefmt='grid', showindex=False))

## Querying livestock performance data
Animal data is found across several different endpoints, however, animal data uses a common ID, *animalid* and we can use that to join results from different queries. For example we can query the livestock performance data and join this to the data we've already queried.

In [ ]:
endpoint = "animals/v1/getLivestockPerformance/"
api_url = f"{base_url}/{endpoint}"
params = {
    "animalId": "1402,1401"
}

response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:    
    data = response.json()
    df = pd.DataFrame(data)
    print(data)
    # Reformat the output in a nice table
    print(tabulate(df, headers='keys', tablefmt='grid'))
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)

